In [3]:
import tensorflow as tf
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Dense
from keras.applications import ResNet50
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np


In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [5]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

def data_augmentation(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_contrast(image, lower=0.2, upper=0.5)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_saturation(image, lower=0.2, upper=0.5)

    image = tf.image.resize(image, [32, 32])

    return image, label

train_dataset = train_dataset.map(data_augmentation)

train_dataset = train_dataset.shuffle(buffer_size=10000).batch(64)
test_dataset = test_dataset.batch(64)


In [6]:
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


model.summary()


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 38, 38, 3) │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 16, 16,    │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 16, 16,    │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 16, 16,    │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 18, 18,    │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 8, 8, 64)  │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 8, 8, 64)  │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 8, 8, 64)  │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 8, 8, 64)  │          0 │ conv2_block1_1_b… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 8, 8, 64)  │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 8, 8, 64)  │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 8, 8, 64)  │          0 │ conv2_block1_2_b… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 8, 8, 256) │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 8, 8, 256) │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 8, 8, 256) │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 8, 8, 256) │      1,024 │ conv2_block1_3_c

 Total params: 25,696,138 (98.02 MB)

 Trainable params: 25,643,018 (97.82 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [7]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [8]:
tf.config.optimizer.set_jit(False)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=test_dataset,
    callbacks=[early_stopping, model_checkpoint]
)


Epoch 1/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.2981 - loss: 2.3553

782/782 ━━━━━━━━━━━━━━━━━━━━ 119s 74ms/step - accuracy: 0.2982 - loss: 2.3546 - val_accuracy: 0.4022 - val_loss: 1.8268
Epoch 2/30
781/782 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5609 - loss: 1.2654

782/782 ━━━━━━━━━━━━━━━━━━━━ 34s 41ms/step - accuracy: 0.5610 - loss: 1.2652 - val_accuracy: 0.5024 - val_loss: 1.5270
Epoch 3/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.6507 - loss: 1.0169

782/782 ━━━━━━━━━━━━━━━━━━━━ 40s 40ms/step - accuracy: 0.6507 - loss: 1.0169 - val_accuracy: 0.6132 - val_loss: 1.1631
Epoch 4/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 30s 37ms/step - accuracy: 0.6961 - loss: 0.8879 - val_accuracy: 0.6029 - val_loss: 1.2235
Epoch 5/30
781/782 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.7291 - loss: 0.7879

782/782 ━━━━━━━━━━━━━━━━━━━━ 43s 39ms/step - accuracy: 0.7292 - loss: 0.7879 - val_accuracy: 0.6503 - val_loss: 1.0368
Epoch 6/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 31s 38ms/step - accuracy: 0.7542 - loss: 0.7200 - val_accuracy: 0.6319 - val_loss: 1.1426
Epoch 7/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.7718 - loss: 0.6710

782/782 ━━━━━━━━━━━━━━━━━━━━ 36s 44ms/step - accuracy: 0.7718 - loss: 0.6710 - val_accuracy: 0.6620 - val_loss: 1.0126
Epoch 8/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 30s 36ms/step - accuracy: 0.7843 - loss: 0.6288 - val_accuracy: 0.6392 - val_loss: 1.1095
Epoch 9/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 30s 38ms/step - accuracy: 0.8002 - loss: 0.5844 - val_accuracy: 0.6574 - val_loss: 1.0643
Epoch 10/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 29s 36ms/step - accuracy: 0.8150 - loss: 0.5460 - val_accuracy: 0.5824 - val_loss: 1.4719
Epoch 11/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 42s 38ms/step - accuracy: 0.8282 - loss: 0.5143 - val_accuracy: 0.6407 - val_loss: 1.1334
Epoch 12/30
781/782 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.8339 - loss: 0.4861

782/782 ━━━━━━━━━━━━━━━━━━━━ 36s 45ms/step - accuracy: 0.8339 - loss: 0.4861 - val_accuracy: 0.7022 - val_loss: 0.9479
Epoch 13/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 29s 36ms/step - accuracy: 0.8443 - loss: 0.4532 - val_accuracy: 0.6633 - val_loss: 1.1549
Epoch 14/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 30s 36ms/step - accuracy: 0.8532 - loss: 0.4367 - val_accuracy: 0.6612 - val_loss: 1.1759
Epoch 15/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 30s 37ms/step - accuracy: 0.8622 - loss: 0.4041 - val_accuracy: 0.6423 - val_loss: 1.2784
Epoch 16/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 40s 36ms/step - accuracy: 0.8693 - loss: 0.3830 - val_accuracy: 0.6668 - val_loss: 1.2435
Epoch 17/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 41s 36ms/step - accuracy: 0.8747 - loss: 0.3658 - val_accuracy: 0.6830 - val_loss: 1.1369


In [9]:
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Fine-tuned Test Accuracy: {test_accuracy * 100:.2f}%")

157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.7057 - loss: 0.9371
Fine-tuned Test Accuracy: 70.22%
